In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
This code snippet is setting up the necessary tools for building, training, and evaluating a machine learning model using TensorFlow, specifically for image classification tasks. Here’s a breakdown of each part:

1. **Library Imports:**
   - `tensorflow` is a popular library for building and training deep learning models.
   - `numpy` is used for mathematical operations and handling arrays, which are essential in machine learning.
   - `matplotlib.pyplot` is a plotting library for visualizing data, often used to plot images, accuracy, and loss during training.

2. **Imports from `tensorflow.keras`:**
   - `Adam` (from `tensorflow.keras.optimizers`): Adam is an optimization algorithm that adjusts the learning rate during training to improve performance. It’s commonly used for training deep learning models.
   - `SparseCategoricalCrossentropy` (from `tensorflow.keras.losses`): This loss function is used when you’re working with categorical data (e.g., classes of images), but the labels are not one-hot encoded. It's suitable for multi-class classification tasks.
   - `Accuracy` (from `tensorflow.keras.metrics`): This metric keeps track of how accurately the model is classifying images.
   - `VGG16` (from `tensorflow.keras.applications.vgg16`): VGG16 is a pre-trained deep learning model known for its strong performance in image classification tasks. It's often used as a starting point for transfer learning, allowing you to leverage learned features from large datasets (like ImageNet).
   - `ImageDataGenerator` (from `tensorflow.keras.preprocessing.image`): This is a utility for loading and processing images. It can apply transformations, like scaling and flipping, to create more varied training data (data augmentation) and help prevent overfitting.

**Overall,** this setup is preparing to load a pre-trained VGG16 model, generate image data, and then train and evaluate the model on image classification tasks with the `Adam` optimizer, using accuracy as a metric and cross-entropy as the loss function. This setup would be commonly used in tasks like classifying images into categories (e.g., dogs vs. cats, or multiple types of objects).

In [ ]:
img_generator = ImageDataGenerator(
    brightness_range=(0.5,1),
    channel_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255,
    validation_split=0.3
)

In [ ]:
This code sets up an `ImageDataGenerator` object, which is a TensorFlow tool used for data augmentation. Data augmentation is a technique to artificially increase the size and variability of your dataset by applying random transformations to the images. These transformations help the model generalize better by exposing it to variations of the images it will see during training.

Here’s what each parameter does in this `ImageDataGenerator` setup:

1. **`brightness_range=(0.5,1)`**: Randomly adjusts the brightness of images. This range (0.5, 1) means that the brightness of each image will be varied between 50% (darker) and 100% (original brightness). 

2. **`channel_shift_range=0.2`**: Randomly shifts the color channels (RGB) by a value within the specified range. This simulates lighting or color variations in the images, which can make the model more robust to differences in lighting or color.

3. **`horizontal_flip=True`**: Randomly flips images horizontally (left-to-right). This is helpful when the orientation of objects isn’t important for classification (e.g., for images of animals or everyday objects that look the same when mirrored).

4. **`vertical_flip=True`**: Randomly flips images vertically (top-to-bottom). This is generally used when orientation doesn't matter for the classification task (like in aerial or medical images).

5. **`rescale=1./255`**: Scales the pixel values of images from their original range (0-255) to a normalized range (0-1). Neural networks typically perform better with normalized data.

6. **`validation_split=0.3`**: Sets aside 30% of the data for validation, meaning 30% of the images will be used for validation (testing the model's performance during training), and the remaining 70% will be used for training.

**In summary:** This `ImageDataGenerator` applies random transformations to the images, which increases data variety and helps prevent overfitting. The model will learn to handle brightness changes, color variations, flipped orientations, and more. This setup also includes automatic rescaling and a 70-30 split between training and validation data.

In [ ]:
root_dir = r"C:\Users\nikun\Downloads\LP-IV-datasets\LP-IV-datasets\Object Detection(Ass6)\caltech-101-img"


img_generator_flow_train = img_generator.flow_from_directory(
    directory=root_dir,
    target_size=(224,224),
    batch_size=32,
    shuffle=True,
    subset="training"
)

img_generator_flow_valid = img_generator.flow_from_directory(
    directory=root_dir,
    target_size=(224, 224),
    batch_size=32,
    shuffle=True,
    subset="validation"
)

In [ ]:
This code sets up two data generators (`img_generator_flow_train` and `img_generator_flow_valid`) for training and validation data by using `ImageDataGenerator` with the dataset stored in the specified directory (`root_dir`). Here’s what each part does:

1. **`root_dir`**:
   - This variable holds the path to the directory where the image dataset is stored. The dataset is expected to be organized in subdirectories where each subdirectory corresponds to a category or class (e.g., "cats" or "dogs" if it's an animal dataset).

2. **`img_generator.flow_from_directory()`**:
   - `flow_from_directory()` is a method in `ImageDataGenerator` that loads images directly from a directory structure and prepares batches of images. It will apply the data augmentation transformations specified earlier in `img_generator` and automatically split the data into training and validation sets based on the `subset` parameter.
   
   Here are the parameters in detail:

   - **`directory=root_dir`**: Specifies the root directory for the dataset, which is where `ImageDataGenerator` will look for subdirectories representing different classes.

   - **`target_size=(224,224)`**: Resizes all images to 224x224 pixels. This is necessary because deep learning models like VGG16 require input images to have a fixed size. In this case, 224x224 is the standard input size for VGG16.

   - **`batch_size=32`**: Specifies that images will be loaded in batches of 32. Batching speeds up training and allows for efficient use of memory, especially when dealing with large datasets.

   - **`shuffle=True`**: Randomly shuffles the images in each batch. Shuffling helps the model generalize better by avoiding order dependencies in the training data.

   - **`subset="training"` and `subset="validation"`**: Using `subset="training"` for `img_generator_flow_train` means this generator will load 70% of the images for training (as set by `validation_split=0.3`). `subset="validation"` in `img_generator_flow_valid` will load the remaining 30% for validation. This automatic splitting helps to evaluate the model on unseen data during training.

**In summary:** 
- `img_generator_flow_train` will generate batches of augmented training images, resized to 224x224, from the specified directory.
- `img_generator_flow_valid` will generate batches of validation images in the same way but without using them for training. Both generators handle shuffling and batching, streamlining the data input process for training a model.

In [ ]:
This code snippet is displaying a batch of images with their labels from the training data generator (`img_generator_flow_train`). Here's a step-by-step explanation of how it works:

1. **`imgs, labels = next(iter(img_generator_flow_train))`:**
   - `iter(img_generator_flow_train)`: Converts the generator (`img_generator_flow_train`) into an iterator, allowing you to retrieve the next batch of images.
   - `next(...)`: Fetches the next batch from the iterator. Since `img_generator_flow_train` generates batches of images, `next(...)` will return a batch of images (`imgs`) and their corresponding labels (`labels`).
   - After this line runs, `imgs` will contain a batch of images (in this case, 32 images of size 224x224 if `batch_size=32`) and `labels` will contain their associated class labels.

2. **Looping through `imgs` and `labels`:**
   - `for img, labels in zip(imgs, labels)`: Loops through each image (`img`) and its corresponding label (`labels`) in the batch.
   - This `zip` pairs each image with its label, allowing you to process and display each image individually in the loop.

3. **Displaying each image with `plt.imshow(img)`:**
   - `plt.imshow(img)`: Displays the image using `matplotlib`. Each `img` here is one image from the batch, resized to 224x224 pixels and preprocessed based on the earlier data generator settings.
   - `plt.show()`: Renders the image in a new plot, one at a time. This will show each image in a separate window or output cell.

**In summary:** This code pulls a single batch of images from the training data generator, then iterates through each image in the batch, displaying it one by one. This can be helpful to visually confirm that the images are being loaded and augmented as expected.

In [ ]:
model_path = r"C:\Users\nikun\Downloads\LP-IV-datasets\LP-IV-datasets\Object Detection(Ass6)\model_weights.h5"

base_model = VGG16(input_shape=(224, 224, 3),
                   include_top=False,
                   weights='imagenet')

In [ ]:
This code initializes a base model for transfer learning using a pre-trained VGG16 model. Let’s go through it in detail:

1. **`model_path`**:
   - This variable stores the path to the file where you might later save or load model weights (`model_weights.h5`). The `.h5` file format is often used to save the trained weights of a model, allowing you to save the model’s parameters and reuse them later without retraining.

2. **`base_model = VGG16(...)`**:
   - This line loads the VGG16 model, which is a well-known convolutional neural network architecture pre-trained on the ImageNet dataset. VGG16 is popular in image classification and transfer learning tasks because of its strong feature extraction capabilities.

   Here are the parameters used:

   - **`input_shape=(224, 224, 3)`**: This specifies the input shape of the images that the model will process. Each image should be 224x224 pixels, with 3 color channels (RGB). This input size matches the default input size of VGG16.

   - **`include_top=False`**: This option excludes the top (fully connected) layers of the VGG16 model, which were originally designed for classifying 1000 categories in ImageNet. By excluding the top layers, you keep only the convolutional layers, which are useful for extracting features. You can then add your own custom layers to adapt the model to your specific classification task (e.g., for a different number of classes).

   - **`weights='imagenet'`**: This parameter loads pre-trained weights from the ImageNet dataset into the VGG16 model. Using pre-trained weights allows the model to benefit from the features learned on a large dataset, which helps with accuracy, especially when your own dataset is relatively small.

**In summary:** This code loads the VGG16 model with pre-trained weights from ImageNet, excluding the top classification layers. This setup is useful for transfer learning, where you can fine-tune the model for your specific task by adding new layers on top of `base_model`. This way, you retain the useful features the model has already learned while customizing it for your own classes.

In [ ]:
base_model.trainable = False

In [ ]:
This line of code:

```python
base_model.trainable = False
```

sets the `trainable` attribute of `base_model` (the VGG16 model) to `False`. Here’s what this does and why it's useful:

1. **Freezing the Pre-trained Layers**:
   - Setting `trainable = False` "freezes" all the layers in the `base_model`, meaning their weights won’t be updated during training.
   - When a layer is frozen, its weights remain the same as when it was originally trained on the ImageNet dataset (or whichever dataset it was pre-trained on).

2. **Why Freeze the Layers?**
   - **Avoid Overfitting**: Freezing pre-trained layers helps prevent overfitting, especially if you have a small dataset. The pre-trained weights already capture useful features, so adjusting them on a small dataset could lead to overfitting.
   - **Speed Up Training**: By freezing the layers, fewer computations are required during training since only the additional layers (added on top of `base_model`) will be trained. This speeds up the training process.

3. **Transfer Learning Strategy**:
   - In transfer learning, it’s common to freeze the base layers initially and only train the new layers you add on top. This way, you leverage the general features the pre-trained model has learned (like edges, textures, and shapes) and only train the model to recognize patterns specific to your dataset in the new layers.
   - After training the new layers, you can optionally "unfreeze" some of the base model layers and fine-tune them if you have enough data and resources.

**In summary:** Setting `base_model.trainable = False` freezes the pre-trained layers of VGG16, so they act as a fixed feature extractor, and only the new layers on top will be trained. This makes the model training faster and helps prevent overfitting.

In [ ]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(102,activation='softmax')
])

In [ ]:
This code creates a new model (`model`) using TensorFlow's `Sequential` API by adding layers on top of the frozen `base_model`. Here’s what each part does:

1. **`tf.keras.Sequential([...])`**:
   - This creates a sequential model, which means the layers will be added one after another in a linear stack.

2. **`base_model`**:
   - The first layer in this stack is `base_model`, which is the frozen VGG16 model without its top classification layers.
   - Since `base_model.trainable = False`, these layers will act as a fixed feature extractor for the images.

3. **`tf.keras.layers.MaxPooling2D()`**:
   - A `MaxPooling2D` layer performs downsampling by reducing the spatial dimensions (height and width) of the feature maps. This layer helps to further condense the features extracted by `base_model`, focusing on the most important information and reducing computation.
   - Since `base_model` already includes convolutional and max pooling layers, this additional pooling layer serves as an extra downsampling step.

4. **`tf.keras.layers.Flatten()`**:
   - The `Flatten` layer converts the 2D feature maps from the previous layer into a 1D vector. This transformation is required before connecting to fully connected (dense) layers, which expect a 1D input.

5. **`tf.keras.layers.Dense(102, activation='softmax')`**:
   - The `Dense` layer is a fully connected layer with 102 units (neurons), where each unit represents a different class in the dataset.
   - `activation='softmax'` specifies the softmax activation function, which is typically used in the output layer of a classification model to produce a probability distribution across the classes.
   - Here, the model will output a vector of 102 probabilities (one for each class), summing to 1, indicating the likelihood that an image belongs to each class.

**In summary:** This model stacks the VGG16 base (as a feature extractor) with additional pooling and dense layers to adapt it to a new classification task with 102 classes. The final `Dense` layer with softmax activation produces the class probabilities, enabling the model to classify each image into one of the 102 categories.

In [ ]:
adam = Adam(0.001)
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
This code sets up the optimizer, loss function, and evaluation metric for training the model. Here’s a breakdown of each component:

1. **`adam = Adam(0.001)`**:
   - This line initializes the Adam optimizer with a learning rate of 0.001. Adam (Adaptive Moment Estimation) is a popular optimization algorithm that combines the benefits of two other optimizers: AdaGrad (adaptive gradients) and RMSProp (root mean square propagation). Adam dynamically adjusts the learning rate during training, which often leads to faster convergence.
   - The learning rate of 0.001 is a common starting point that balances speed and stability in training.

2. **`model.compile(...)`**:
   - `compile()` is where you specify the optimizer, loss function, and evaluation metrics to use during training. It prepares the model for training by configuring these settings.

   - **`optimizer='adam'`**: Specifies Adam as the optimization algorithm. By providing `'adam'` as a string instead of using the `adam` object defined earlier, TensorFlow will use the Adam optimizer with default parameters (including a learning rate of 0.001).
   
   - **`loss='categorical_crossentropy'`**: Sets the loss function to categorical cross-entropy, which is commonly used in multi-class classification tasks where the labels are one-hot encoded. Categorical cross-entropy calculates the difference between the predicted class probabilities and the true class probabilities, penalizing the model when it misclassifies.
   
   - **`metrics=['accuracy']`**: Specifies that the model should track accuracy during training and evaluation. Accuracy is a common metric for classification tasks, showing the proportion of correctly classified images out of the total.

**In summary:** This code configures the model for training with the Adam optimizer (learning rate 0.001), categorical cross-entropy as the loss function (suitable for multi-class classification), and accuracy as the evaluation metric. This setup will help the model optimize its predictions and provide a straightforward measure of performance.

In [ ]:
model.fit(img_generator_flow_train, 
          validation_data=img_generator_flow_valid, 
          steps_per_epoch=5,
          epochs = 5)

In [ ]:
This code begins the model training process using the `.fit()` function, which iteratively adjusts the model’s weights to improve its predictions. Here’s a detailed look at each parameter:

1. **`img_generator_flow_train`**:
   - This is the training data generator created with `ImageDataGenerator`. It supplies batches of training images and labels, including any augmentations applied (like flips and brightness adjustments).
   - `img_generator_flow_train` continuously feeds batches of training images to the model during training.

2. **`validation_data=img_generator_flow_valid`**:
   - This specifies the validation data generator, `img_generator_flow_valid`, which provides batches of validation images and labels.
   - The validation data is used to evaluate the model's performance on unseen data after each epoch, helping monitor the model's generalization.

3. **`steps_per_epoch=5`**:
   - This parameter defines the number of batches the model will process in each epoch. 
   - Setting `steps_per_epoch=5` means the model will process 5 batches of 32 images (the batch size) in each epoch, resulting in a total of \(5 \times 32 = 160\) training images per epoch.
   - Adjusting `steps_per_epoch` is useful when working with large datasets or limited computational resources. If the dataset is small, you might choose to set `steps_per_epoch` so that all images in the training set are seen in each epoch.

4. **`epochs=5`**:
   - The `epochs` parameter sets the number of complete training cycles. An epoch represents one pass through the specified training batches.
   - Setting `epochs=5` means the model will train for 5 complete cycles, with evaluation on the validation data after each epoch.

**In summary:** This code initiates training for 5 epochs, with each epoch processing 5 batches of 32 images from the training data and evaluating on the validation data afterward. The results after each epoch provide feedback on both training and validation performance, helping you track the model's learning and detect any signs of overfitting.

In [ ]:
plt.plot(model.history.history['accuracy'], label='train_accuracy')
plt.plot(model.history.history['val_accuracy'],label='test_accuracy')
plt.legend()

In [ ]:
This code visualizes the training and validation accuracy over the epochs, which can help you understand the model's performance trends.

Here’s a breakdown of each part:

1. **`model.history.history['accuracy']`**:
   - `model.history.history` is a dictionary containing the metrics recorded during training. 
   - `model.history.history['accuracy']` retrieves the accuracy values for the training data after each epoch. These values reflect how well the model performed on the training set over time.

2. **`model.history.history['val_accuracy']`**:
   - `model.history.history['val_accuracy']` retrieves the accuracy values for the validation data after each epoch. These values indicate how well the model generalizes to unseen data, helping you monitor for signs of overfitting.

3. **`plt.plot(..., label='train_accuracy')`**:
   - This line plots the training accuracy over the epochs. `label='train_accuracy'` gives the line a label, which will show up in the legend.

4. **`plt.plot(..., label='test_accuracy')`**:
   - This line plots the validation (test) accuracy over the epochs, similarly labeled.

5. **`plt.legend()`**:
   - Adds a legend to the plot, showing labels for `train_accuracy` and `test_accuracy` to distinguish between the two lines.

**In summary:** This code produces a plot of training and validation accuracy over the epochs. The resulting graph can help you:
   - Assess if the model is improving (accuracy should generally increase with each epoch).
   - Identify overfitting (if validation accuracy plateaus or decreases while training accuracy continues to improve).

In [ ]:
base_model.trainable = True

model.compile(loss='categorical_crossentropy',metrics=['categorical_accuracy'],optimizer='adam')